In [70]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from IPython.display import display
import numpy as np

random_state = 42

In [71]:
ham = pd.read_csv("./resources/features-enron.csv")
ham = ham.drop("Unnamed: 0", axis=1)
ham.head()

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,Phishy,URLs,id
0,False,0,0,7bit,0,False,False,False,False,False,0,False,0,1
1,False,0,0,7bit,0,False,False,False,False,False,0,False,0,2
2,False,0,0,7bit,0,False,False,False,False,False,0,False,0,3
3,False,0,0,7bit,0,False,False,False,False,False,0,False,0,4
4,False,0,0,quoted-printable,0,False,False,False,False,False,0,False,0,5


In [72]:
spam = pd.read_csv("./resources/features-phishing.csv")
spam = spam.drop("Unnamed: 0", axis=1)
spam.head()


,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,Phishy,URLs,id
0,False,0,0,none,0,False,True,False,False,True,0,True,2,1
1,False,0,0,7bit,0,False,True,False,False,False,0,True,2,2
2,False,0,0,7bit,0,False,True,False,False,False,0,True,2,3
3,False,0,0,8bit,0,False,True,False,False,True,0,True,3,4
4,False,0,0,8bit,0,False,True,False,False,True,0,True,3,5


# Get X and y, before test/train split

In [91]:
X = pd.concat([spam, ham])
y = X["Phishy"] 

X = X.drop("Phishy", axis=1)
if "id" in X.columns:
    X = X.drop("id", axis=1)

print("X: ", X.shape, " y: ", y.shape)
display(X.head())

y.head()
print("datatypes X \n{0} \ny \n{1}".format(X.dtypes, y.dtypes))

dEncoding = {}
i = 0
for enc in X["Encoding"].unique():
    dEncoding[enc] = i
    i+=1




X:  (6535, 12)  y:  (6535,)


,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,URLs
0,False,0,0,none,0,False,True,False,False,True,0,2
1,False,0,0,7bit,0,False,True,False,False,False,0,2
2,False,0,0,7bit,0,False,True,False,False,False,0,2
3,False,0,0,8bit,0,False,True,False,False,True,0,3
4,False,0,0,8bit,0,False,True,False,False,True,0,3


datatypes X 
@ in URLs               bool
Attachments            int64
Css                    int64
Encoding              object
External Resources     int64
Flash content           bool
HTML content            bool
Html Form               bool
Html iFrame             bool
IPs in URLs             bool
Javascript             int64
URLs                   int64
dtype: object 
y 
bool


# clean up 'Encoding' column

In [92]:
X = X.replace({"Encoding": dEncoding})
X["Encoding"].unique()



array([0, 1, 2, 3, 4, 5, 6, 7])

# Split into train and test

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_state)

display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,URLs
687,False,0,0,1,0,False,False,False,False,False,0,0
596,False,0,0,0,0,False,True,False,False,False,0,2
1972,False,0,0,0,0,False,False,False,False,False,0,0
1068,False,0,0,0,0,False,True,False,False,False,0,24
1942,False,0,0,1,0,False,True,False,False,False,0,4


687     False
596      True
1972    False
1068     True
1942     True
Name: Phishy, dtype: bool

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,URLs
3180,False,0,0,1,0,False,False,False,False,False,0,0
604,False,0,0,1,0,False,False,False,False,False,0,0
595,False,0,0,1,0,False,False,False,False,False,0,0
233,False,0,0,0,0,False,True,False,False,True,0,2
1472,False,0,0,0,0,False,True,False,False,False,0,0


3180    False
604     False
595     False
233      True
1472    False
Name: Phishy, dtype: bool

# Hyperparameter Tuning

In [94]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2010, num = 20)]
n_estimators = np.arange(100, 2000, 100)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(1, 50, 2)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
# bound = 5000
# rf_random.fit(X_train[:bound], y_train[:bound])
rf_random.fit(X_train, y_train)

print("best est   : ", rf_random.best_estimator_)
print("best params: ", rf_random.best_params_)
print("best score : ", rf_random.best_score_)

{'bootstrap': [True, False],
 'max_depth': [1,
               3,
               5,
               7,
               9,
               11,
               13,
               15,
               17,
               19,
               21,
               23,
               25,
               27,
               29,
               31,
               33,
               35,
               37,
               39,
               41,
               43,
               45,
               47,
               49],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
       1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900])}
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  7.5min finished


best est   :  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=49, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
best params:  {'n_estimators': 1700, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 49, 'bootstrap': True}
best score :  0.9602558245774326


# Model w tuning

In [95]:
rf_tuned = rf_random.best_estimator_

In [96]:
rf_tuned.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=49, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [97]:
rf_tuned.score(X_test,y_test)

0.9629114510894762

In [98]:
from sklearn.metrics import classification_report
print(classification_report(rf_tuned.predict(X_test), y_test, digits=5))

              precision    recall  f1-score   support

       False    0.95848   0.98384   0.97099      1361
        True    0.97105   0.92714   0.94859       796

    accuracy                        0.96291      2157
   macro avg    0.96477   0.95549   0.95979      2157
weighted avg    0.96312   0.96291   0.96272      2157



# Model w/o tuning

In [47]:
rf_notuned = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=random_state)

In [48]:
rf_notuned.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [49]:
rf_notuned.score(X_test, y_test)

0.9601298099211868

# Tuned Model Results

In [99]:
from sklearn.metrics import confusion_matrix
y_pred = rf_tuned.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# precision - what proportion of flagged phishing emails were truely phishing?
precision = tp / (tp + fp)
# recall - how many of the phishing emails were correctly identified?
recall = tp / (tp + fn)
# f1 statistic - combination of precision and recall
f1 = 2 * precision * recall / (precision + recall)
# accuracy - what proportion were identified correctly
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("precision                              [tp / (tp + fp)] = %f" % precision)
print("recall                                 [tp / (tp + fn)] = %f" % recall)
print("f1        2 * precision * recall / (precision + recall) = %f" % f1)
print("accuracy                 (tp +tn) / (tp + tn + fp + fn) = %f" % accuracy)

# print(len(y_pred))
# print(conf_mat)
# conf_mat / len(y_pred)

precision                              [tp / (tp + fp)] = 0.927136
recall                                 [tp / (tp + fn)] = 0.971053
f1        2 * precision * recall / (precision + recall) = 0.948586
accuracy                 (tp +tn) / (tp + tn + fp + fn) = 0.962911


# Save Model

In [100]:
from joblib import dump, load
model_filename = "RandomForestModel.dat"
best_params_filename = "RandomForestModel.params"
dump(rf_tuned, model_filename)
dump(rf_random.best_params_, best_params_filename)

dump(dEncoding, "encoding.bin")



['encoding.bin']

# Load Model 

In [83]:
rf_tuned_saved = load(model_filename)
y_pred_saved = rf_tuned_saved.predict(X_test)

display(sum(y_pred != y_pred_saved))

rf_tuned_best_params = load(best_params_filename)
print("best_params: ", rf_tuned_best_params)

0

best_params:  {'n_estimators': 1800, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 37, 'bootstrap': True}
